In [ ]:
from parse_code import *
from parse_files import *
from print_contents import *
import faiss
import json
from sentence_transformers import SentenceTransformer
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
import numpy as np
from pathlib import Path
import torch
import subprocess
import os
#torch.cuda.is_available()

In [20]:
def run_command(command):
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    assert result.returncode == 0, f"Command '{command}' failed with error: {result.stderr}"

In [ ]:
# system calls -- I wrote some cmdline scripts to do the parsing
call_1 = "uv run parse_code.py ../App/python-server/ --replace-source ../App --replace-target ./App -o ./data/backend_code.txt"
call_2 = "uv run print_contents.py -d ../ -r --exclude-dirs exclude_dirs.txt --exclude-files exclude_files.txt -o ./data/project_structure.txt"
call_3 = "uv run parse_files.py --exclude-dirs exclude_dirs.txt --exclude-files exclude_files.txt -o ./data/project_files.txt ../"

In [24]:
run_command(call_1)
run_command(call_2)
run_command(call_3)

In [26]:
code_data = json.load(open('./data/backend_code.txt'))

In [7]:
print(f"Model's maximum sequence length: {SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2').max_seq_length}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model's maximum sequence length: 256


In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"},
    
)

True